# Topic modeling

In this notebook, we convert screening arrays into documents:
* Document: fragment
* Word: protein
* Topic: group of proteins

## Convert screening to documents

In [1]:
import pandas as pd
import csv
import collections
import numpy as np
import os
import shutil
import json
from socket import socket
import blitzgsea as blitz
import joblib
import h5py
import topicwizard
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer
from sklearn.neighbors import NearestNeighbors
from itertools import combinations
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from lol import LOL
from tabpfn import TabPFNClassifier
from sklearn.model_selection import StratifiedShuffleSplit

MAX_PROTEIN_COUNTS = 40 # 40: 10% of fragments
MIN_PROTEIN_COUNTS = 4 # 4: 1% of fragments
MAX_PROTEINS_PER_FRAGMENTS = 100
MIN_PROTEINS_PER_FRAGMENTS = 20
TFIDF = False

RESULTS_DIR = "../results"

if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)

with socket() as s:
    s.bind(('',0))
    PORT = s.getsockname()[1]
print("PORT", PORT)

output_folder = "topwiz_maxpxf{0}_minp{1}_maxp{2}".format(MAX_PROTEINS_PER_FRAGMENTS, MIN_PROTEIN_COUNTS, MAX_PROTEIN_COUNTS)

if os.path.exists("{0}/{1}".format(RESULTS_DIR, output_folder)):
    shutil.rmtree("{0}/{1}".format(RESULTS_DIR, output_folder))
os.mkdir("{0}/{1}".format(RESULTS_DIR, output_folder))

metadata = {
    "MAX_PROTEIN_COUNTS": MAX_PROTEIN_COUNTS,
    "MIN_PROTEIN_COUNTS": MIN_PROTEIN_COUNTS,
    "MAX_PROTEINS_PER_FRAGMENTS": MAX_PROTEINS_PER_FRAGMENTS,
    "MIN_PROTEINS_PER_FRAGMENTS": MIN_PROTEINS_PER_FRAGMENTS,
    "TFIDF": TFIDF
}
with open(os.path.join(RESULTS_DIR, output_folder, "metadata.json"), "w") as f:
    json.dump(metadata, f)

def get_X_from_esm():
    with h5py.File("../data/pid2seq_esm1b.h5", "r") as f:
        the_pids = [x.decode("utf-8") for x in f["Keys"][:]]
        X = f["Values"][:]
        the_pids_idxs = dict((k,i) for i,k in enumerate(the_pids))
    return X, the_pids, the_pids_idxs

def get_X_from_bioteque(metapath_name="gene_has_cmp"):
    with h5py.File("../data/bioteque/{0}/GEN_emb.h5".format(metapath_name), "r") as f:
        X = f["m"][:]
    with open("../data/bioteque/{0}/GEN_ids.txt".format(metapath_name), "r") as f:
        the_pids = []
        for l in f:
            the_pids += [l.rstrip()]
    the_pids_idxs = dict((k,i) for i,k in enumerate(the_pids))
    return X, the_pids, the_pids_idxs

def get_X_merged(n_components=300):
    X_1, the_pids_1, the_pids_idxs_1 = get_X_from_bioteque("gene_has_cmp")
    X_2, the_pids_2, the_pids_idxs_2 = get_X_from_esm()
    the_pids = sorted(set(the_pids_1).intersection(the_pids_2))
    R = []
    for pid in the_pids:
        r = [x for x in X_1[the_pids_idxs_1[pid]]] + [x for x in X_2[the_pids_idxs_2[pid]]]
        R += [r]
    X = np.array(R)
    X = PCA(n_components=n_components).fit_transform(X)
    the_pids_idxs = dict((k,i) for i,k in enumerate(the_pids))
    return X, the_pids, the_pids_idxs

X_emb, emb_pids, emb_pids_idxs = get_X_merged()

df = pd.read_csv("../data/cemm_primary_data.tsv", sep="\t")
df = df[df["UniqPeptides"] >= 2]
db = pd.read_csv("../data/cemm_primary_hit_data.tsv", sep="\t")

fids = sorted(set(df["FragID"]))
fid2smi = pd.read_csv("../data/fid2can_fff_all.tsv", sep="\t")
fid2smi = fid2smi[fid2smi["fid"].isin(fids)].sort_values("fid")
fid2smi_dict = {}
for r in fid2smi.values:
    fid2smi_dict[r[0]] = r[1]

pid2name = {}
with open("../data/pid2name_primary_all.tsv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for r in reader:
        pid2name[r[0]] = r[1]

protein_counts = collections.defaultdict(int)
for r in db["UniProtID"].tolist():
    protein_counts[r] += 1
protein_counts = sorted(protein_counts.items(), key=lambda x: -x[1])

fragment_counts = collections.defaultdict(int)
for r in db["FragID"].tolist():
    fragment_counts[r] += 1
fragment_counts = sorted(fragment_counts.items(), key=lambda x: -x[1])

df = pd.read_csv("../data/cemm_primary_data.tsv", sep="\t")
db = pd.read_csv("../data/cemm_primary_hit_data.tsv", sep="\t")

protein_counts = collections.defaultdict(int)
for r in db["UniProtID"].tolist():
    protein_counts[r] += 1

max_counts = MAX_PROTEIN_COUNTS
pids = sorted(set(db["UniProtID"]))
pids = [x for x in pids if protein_counts[x] <= max_counts]
min_counts = MIN_PROTEIN_COUNTS
pids = [x for x in pids if protein_counts[x] >= min_counts]
pids = sorted(set(pids).intersection(emb_pids))

emb_idxs = []
for pid in pids:
    emb_idxs += [emb_pids_idxs[pid]]
X_emb = X_emb[emb_idxs]

pid2idx = dict((k,i) for i,k in enumerate(pids))
pids_set = set(pids)

fids = sorted(set(df["FragID"]))
fid2idx = dict((k,i) for i,k in enumerate(fids))

fid2smi_ = pd.read_csv("../data/fid2can_fff_all.tsv", delimiter="\t")
fid2smi = {}
for r in fid2smi_.values:
    if r[0] in fid2idx:
        fid2smi[r[0]] = r[1]

Y_pro = np.full((len(fids), len(pids)), np.nan)

for r in df[["FragID", "UniProtID", "UniqPeptides", "RankRelative", "Log2FC", "Log2FCMedian", "p", "pAdj"]].values:
    if r[1] not in pids_set:
        continue
    i = fid2idx[r[0]]
    j = pid2idx[r[1]]
    unique_peptides = float(r[2])
    rank_relative = float(r[3])
    log2fc = float(r[4])
    log2fc_median = float(r[5])
    p_value = float(r[6])
    p_value_adj = float(r[7])
    if unique_peptides < 2:
        continue
    if np.isnan(p_value):
        continue
    if np.isnan(log2fc):
        continue
    Y_pro[i,j] = log2fc

R_pro = np.array(Y_pro)
R_pro[np.isnan(R_pro)] = -999

def read_annotations(annotations_file):
    d = collections.defaultdict(list)
    with open(annotations_file, "r") as f:
        reader = csv.reader(f, delimiter="\t")
        for r in reader:
            if r[0] not in pid2name:
                continue
            d[r[1]] += [pid2name[r[0]]]
    d = dict((k, set(v)) for k,v in d.items() if len(v) >= 5)
    return d

cc_annotations = read_annotations("../data/msigdb_gocc.tsv")
mf_annotations = read_annotations("../data/msigdb_gomf.tsv")

joblib.dump(cc_annotations, os.path.join(RESULTS_DIR, output_folder, "msigdb_gocc.joblib"))
joblib.dump(mf_annotations, os.path.join(RESULTS_DIR, output_folder, "msigdb_gomf.joblib"))

/Users/mduranfrigola/miniconda3/envs/topicmodel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PORT 64138


['../results/topwiz_maxpxf100_minp4_maxp40/msigdb_gomf.joblib']

In [2]:
%%capture

gsea_prefiltering_file = os.path.join(RESULTS_DIR, "gsea_prefiltering_minp{0}_maxp{1}.joblib".format(MIN_PROTEIN_COUNTS, MAX_PROTEIN_COUNTS))

if os.path.exists(gsea_prefiltering_file):
    gsea_prefiltering_results = joblib.load(gsea_prefiltering_file)
else:
    gsea_prefiltering_results = []
    for i in range(R_pro.shape[0]):
        v = np.array(R_pro[i,:])
        mask = v != -999
        pids_ = [pids[i] for i in range(len(v)) if mask[i]]
        v_ = PowerTransformer().fit_transform(v[mask].reshape(-1,1))[:,0]
        genes_ = [pid2name[p] for p in pids_]
        signature = pd.DataFrame({0: genes_, 1: v_})
        cc_ = dict((k,list(v.intersection(genes_))) for k,v in cc_annotations.items())
        mf_ = dict((k,list(v.intersection(genes_))) for k,v in mf_annotations.items())
        cc_ = dict((k,v) for k,v in cc_.items() if len(v) >= 5)
        mf_ = dict((k,v) for k,v in mf_.items() if len(v) >= 5)
        result_cc = blitz.gsea(signature, cc_, anchors=4, permutations=2000, processes=4, signature_cache=True)
        result_mf = blitz.gsea(signature, mf_, anchors=4, permutations=2000, processes=4, signature_cache=True)
        gsea_prefiltering_results += [(fids[i], signature, result_cc, result_mf)]
    joblib.dump(gsea_prefiltering_results, gsea_prefiltering_file)

In [3]:
def do_documents_list(min_proteins_per_fragment):
    gsea_pval = 0.05
    documents_list = []
    cap = 250
    hits = []
    for fid in fids:
        pids_ = db[db["FragID"] == fid]["UniProtID"]
        pids_ = set(pids_).intersection(pids)
        genes_ = [pid2name[p] for p in pids_]
        hits += [set(genes_)]
    gid2idx = dict((pid2name[k],i) for i,k in enumerate(pids))
    for i in range(R_pro.shape[0]):
        gs = gsea_prefiltering_results[i]
        cc = gs[2]
        cc = cc[cc["pval"] < gsea_pval]
        mf = gs[3]
        mf = mf[mf["pval"] < gsea_pval]
        cc_genes = [g for x in cc["leading_edge"].tolist() for g in x.split(",")]
        mf_genes = [g for x in mf["leading_edge"].tolist() for g in x.split(",")]
        le = list(cc_genes + mf_genes)
        le = set([x for x in le if x != ""])
        v = R_pro[i,:]
        idxs = np.argsort(v)[::-1][:cap]
        pids_ = [pids[i] for i in idxs]
        genes_base = [pid2name[p] for p in pids_]
        genes_ = [g for g in genes_base if g in le]
        if len(genes_) > MAX_PROTEINS_PER_FRAGMENTS:
            genes_ = genes_[:MAX_PROTEINS_PER_FRAGMENTS]
        else:
            if len(genes_) < min_proteins_per_fragment:
                missing = min_proteins_per_fragment - len(genes_)
                ht = hits[i]
                exclusive_ht = ht.difference(le)
                if len(exclusive_ht) >= missing:
                    eh_list = list(exclusive_ht)
                    le_list = list(le)
                    eh_idxs = [gid2idx[k] for k in eh_list]
                    le_idxs = [gid2idx[k] for k in le_list]
                    nn = NearestNeighbors(n_neighbors=3)
                    nn.fit(X_emb[le_idxs])
                    I, D = nn.kneighbors(X_emb[eh_idxs])
                    dists = np.mean(D, axis=1)
                    idxs_ = np.argsort(dists)[:missing]
                    sel_eh = set([eh_list[idx] for idx in idxs_])
                    genes_ = [g for g in genes_base if g in le or g in sel_eh]
                else:
                    genes_ = [g for g in genes_base if g in le or g in ht]
                    missing = min_proteins_per_fragment - len(genes_)
                    eb_list = [g for g in genes_base if g not in genes_]
                    eb_idxs = [gid2idx[k] for k in eb_list]
                    le_list = list(le)
                    le_idxs = [gid2idx[k] for k in le_list]
                    nn = NearestNeighbors(n_neighbors=3)
                    nn.fit(X_emb[le_idxs])
                    I, D = nn.kneighbors(X_emb[eb_idxs])
                    dists = np.mean(D, axis=1)
                    idxs_ = np.argsort(dists)[:missing]
                    sel_eb = set([eb_list[idx] for idx in idxs_])
                    genes_ = [g for g in genes_base if g in le or g in ht or g in sel_eb]
            else:
                pass
        documents_list += [genes_]
    return documents_list

def create_documents():
    f0 = open("{1}/{0}/gene_docs.txt".format(output_folder, RESULTS_DIR), "w")
    f1 = open("{1}/{0}/fids.txt".format(output_folder, RESULTS_DIR), "w")
    for i, g in enumerate(documents_list):
        f0.write(" ".join(g) + "\n")
        f1.write(fids[i] + "\n")
    f0.close()
    f1.close()

documents_list = do_documents_list(MIN_PROTEINS_PER_FRAGMENTS)
create_documents()

## Non-negative matrix factorization

In [4]:
%%capture

from itertools import combinations
from scipy import spatial

num_topic_trials = [10, 11, 12, 13, 14, 15, 16]

from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
import joblib

def tokenizer(x):
    return x.split(" ")

if not TFIDF:
    vectorizer = CountVectorizer(lowercase=False, tokenizer=tokenizer)
else:
    vectorizer = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

def get_texts():
    with open("{1}/{0}/gene_docs.txt".format(output_folder, RESULTS_DIR), "r") as f:
        texts = [x.rstrip() for x in f.readlines()]
    return texts

def get_doc_names():
    with open("{1}/{0}/fids.txt".format(output_folder, RESULTS_DIR), "r") as f:
        doc_names = [x.rstrip() for x in f.readlines()]
    return doc_names

texts = get_texts()
doc_names = get_doc_names()
 
def fit_topic_pipeline(num_topics):
    mdl = NMF(n_components=num_topics, max_iter=1000)
    topic_pipeline = Pipeline(
       [
          ("vec", vectorizer),
          ("mdl", mdl),
       ]
    )
    texts = get_texts()
    doc_names = get_doc_names()
    topic_pipeline.fit(texts)
    return topic_pipeline

def calculate_coherence(term_rankings):
    gid2idx = dict((pid2name[k],i) for i,k in enumerate(pids))
    print(len(gid2idx))
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        pair_scores = []
        for pair in combinations(term_rankings[topic_index], 2):
            t0 = pair[0]
            t1 = pair[1]
            x0 = X_emb[gid2idx[t0]]
            x1 = X_emb[gid2idx[t1]]
            pair_scores.append(1-spatial.distance.cosine(x0,x1))
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    return overall_coherence / len(term_rankings)

def get_descriptor(all_terms, H, topic_index, top):
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( all_terms[term_index] )
    return top_terms

def screen_topics(topic_pipelines):
    topic_models = []
    for i, tp in enumerate(topic_pipelines):
        k = num_topic_trials[i]
        W = tp.transform(texts)
        H = tp["mdl"].components_
        topic_models += [(k, W, H)]
    k_values = []
    coherences = []
    terms = topicwizard.prepare.utils.get_vocab(tp["vec"])
    for (k,W,H) in topic_models:
        term_rankings = []
        for topic_index in range(k):
            term_rankings.append(get_descriptor(terms, H, topic_index, 10))
        k_values.append(k)
        coherences.append(calculate_coherence(term_rankings))
        print("K=%02d: Coherence=%.4f" % (k, coherences[-1]))
    return coherences

topic_pipelines = [fit_topic_pipeline(n) for n in num_topic_trials]
coherences = screen_topics(topic_pipelines)
        
NUM_TOPICS = 10
topic_pipeline = fit_topic_pipeline(num_topics=NUM_TOPICS)
vec = topic_pipeline["vec"]
mdl = topic_pipeline["mdl"]
    
data_for_wizard = {"corpus": texts, "pipeline": topic_pipeline, "document_names": doc_names}

joblib.dump(data_for_wizard, os.path.join(RESULTS_DIR, output_folder, "data_for_wizard.joblib"))

## Topic Wizard

In [5]:
%%capture

import topicwizard

data_for_wizard = joblib.load(os.path.join(RESULTS_DIR, output_folder, "data_for_wizard.joblib"))

topicwizard.visualize(**data_for_wizard, port=PORT)

In [6]:
topic_names = ["Topic {0}".format(i) for i in range(NUM_TOPICS)]

topic_data = {
    "document_names": doc_names,
    "corpus": texts,
    "vectorizer": vec,
    "topic_model": mdl,
    "topic_names": topic_names
}

print(topic_names)

joblib.dump(topic_data, os.path.join(RESULTS_DIR, output_folder, "topic_data.joblib"))

['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7', 'Topic 8', 'Topic 9']


['../results/topwiz_maxpxf100_minp4_maxp40/topic_data.joblib']

In [7]:
topic_data = joblib.load(os.path.join(RESULTS_DIR, output_folder, "topic_data.joblib"))

document_names = topic_data["document_names"]
corpus = topic_data["corpus"]
vectorizer = topic_data["vectorizer"]
topic_model = topic_data["topic_model"]
topic_names = topic_data["topic_names"]

vocab = topicwizard.prepare.utils.get_vocab(vectorizer)
document_term_matrix, document_topic_matrix, topic_term_matrix = topicwizard.prepare.utils.prepare_transformed_data(vectorizer, topic_model, corpus)
topic_importances, term_importances, topic_term_importances = topicwizard.prepare.topics.topic_importances(topic_term_matrix, document_term_matrix, document_topic_matrix)

word_pos = topicwizard.prepare.words.word_positions(topic_term_matrix)
R = []
for i,t in enumerate(vocab):
    r = [t] + [term_importances[i]] + [word_pos[0][i], word_pos[1][i]] + [x for x in topic_term_importances[:,i]]
    R += [r]

d_ = pd.DataFrame(R, columns=["gene_name", "importance"] + ["proj_x", "proj_y"] + topic_names)
d_.to_csv(os.path.join(RESULTS_DIR, output_folder, "ProteinHasTopic.csv"), index=False)

R = []
for i,t in enumerate(topic_names):
    R += [t] + [topic_importances[i]]

from rdkit import Chem

def has_tertiary_amine(smiles):
    mol = Chem.MolFromSmiles(smiles)
    patt = Chem.MolFromSmarts("[NX3]([CX4])([CX4])[CX4]")
    t = mol.HasSubstructMatch(patt)
    if t:
        return 1
    else:
        return 0
    
doc_pos = topicwizard.prepare.documents.document_positions(document_term_matrix)

R = []
for i in range(len(doc_names)):
    fid_ = doc_names[i]
    smi_ = fid2smi[fid_]
    r = [fid_, smi_, has_tertiary_amine(smi_)] + [doc_pos[0][i], doc_pos[1][i]] + [x for x in document_topic_matrix[i]]
    R += [r]

d_ = pd.DataFrame(R, columns=["fid", "smiles", "tert_amine"] + ["proj_x", "proj_y"] + topic_names)
d_.to_csv(os.path.join(RESULTS_DIR, output_folder, "FragmentHasTopic.csv"), index=False)

processed_topicwizard_data = {
    "vocab": vocab,
    "document_term_matrix": document_term_matrix,
    "document_topic_matrix": document_topic_matrix,
    "topic_term_matrix": topic_term_matrix,
    "topic_importances": topic_importances,
    "term_importances": term_importances,
    "topic_term_importances": topic_term_importances,
    "document_topic_importances": topicwizard.prepare.documents.document_topic_importances(document_topic_matrix)
}

joblib.dump(processed_topicwizard_data, os.path.join(RESULTS_DIR, output_folder, "processed_topicwizard_data.joblib"))


['../results/topwiz_maxpxf100_minp4_maxp40/processed_topicwizard_data.joblib']

In [8]:
%%capture

W = document_topic_matrix

sns.clustermap(pd.DataFrame(W, index=document_names, columns=topic_names), cmap="YlGnBu")

plt.savefig(os.path.join(RESULTS_DIR, output_folder, "heatmap_docs_topic.png"), dpi=600)

In [9]:
%%capture

H = topic_term_importances.T
sns.clustermap(pd.DataFrame(H, index=vocab, columns=topic_names), cmap="YlGnBu")

plt.savefig(os.path.join(RESULTS_DIR, output_folder, "heatmap_words_topic.png"), dpi=600)

In [10]:
%%capture

mf_library = mf_annotations
cc_library = cc_annotations

genes = set(vocab)

mf_sets = {}
for k,v in mf_library.items():
    g = genes.intersection(v)
    if len(g) < 5:
        continue
    mf_sets[k] = list(g)

cc_sets = {}
for k,v in cc_library.items():
    g = genes.intersection(v)
    if len(g) < 5:
        continue
    cc_sets[k] = list(g)

signatures = []
for i, t in enumerate(topic_names):
    v = topic_term_importances[i,:]
    g = vocab
    R = []
    for j in range(len(v)):
        R += [[g[j], v[j]]]
    signature = pd.DataFrame(R)
    signatures += [signature]

gseas_mf = []
for i, t in enumerate(topic_names):
    result = blitz.gsea(signatures[i], mf_sets)
    gseas_mf += [result]

gseas_cc = []
for i, t in enumerate(topic_names):
    result = blitz.gsea(signatures[i], cc_sets)
    gseas_cc += [result]

gsea_results = {
    "topic_names": topic_names,
    "signatures": signatures,
    "cc_sets": cc_sets,
    "mf_sets": mf_sets,
    "gseas_cc": gseas_cc,
    "gseas_mf": gseas_mf
}

joblib.dump(gsea_results, os.path.join(RESULTS_DIR, output_folder, "gsea_results.joblib"))

gsea_plots_folder = os.path.join(RESULTS_DIR, output_folder, "gsea_plots")
if os.path.exists(gsea_plots_folder):
    shutil.rmtree(gsea_plots_folder)
os.mkdir(gsea_plots_folder)

for i,t in enumerate(topic_names):
    fig_table = blitz.plot.top_table(signatures[i], cc_sets, gseas_cc[i], n=15)
    fig_table.savefig(os.path.join(gsea_plots_folder, "{0}_cc.png".format(t.replace(" ", "_"))), bbox_inches='tight', transparent=False, facecolor="white")
    
for i,t in enumerate(topic_names):
    fig_table = blitz.plot.top_table(signatures[i], mf_sets, gseas_mf[i], n=15)
    fig_table.savefig(os.path.join(gsea_plots_folder, "{0}_mf.png".format(t.replace(" ", "_"))), bbox_inches='tight', transparent=False, facecolor="white")

In [11]:
%%capture

import gseapy
from tqdm import tqdm

def enrichr_analysis(num_proteins_per_topic):
    enrs = []
    for i in tqdm(range(NUM_TOPICS)):
        d = topicwizard.prepare.topics.calculate_top_words(i, num_proteins_per_topic, 1, term_importances, topic_term_importances, vocab)
        gene_list = list(d["word"])
        enr = gseapy.enrich(gene_list=gene_list, gene_sets=[cc_sets, mf_sets], background=[pid2name[p] for p in pids])
        enrs += [enr]
    joblib.dump(enrs, os.path.join(RESULTS_DIR, output_folder, "enrichr_results_{0}.joblib".format(num_proteins_per_topic)))
    enrichr_results_folder = os.path.join(RESULTS_DIR, output_folder, "enrichr_results_{0}".format(num_proteins_per_topic))
    if os.path.exists(enrichr_results_folder):
        shutil.rmtree(enrichr_results_folder)
    os.mkdir(enrichr_results_folder)
    for i in range(len(topic_names)):
        d_ = enrs[i].results
        d_.to_csv(os.path.join(enrichr_results_folder, "topic_{0}.csv".format(i)), index=False)
    return enrs

enrs_0 = enrichr_analysis(5)
enrs_1 = enrichr_analysis(10)
enrs_2 = enrichr_analysis(25)

In [12]:
%%capture

def plot_enrichr(enrs, num_proteins_per_topic):
    enrichr_plots_folder = os.path.join(RESULTS_DIR, output_folder, "enrichr_plots_{0}".format(num_proteins_per_topic))
    if os.path.exists(enrichr_plots_folder):
        shutil.rmtree(enrichr_plots_folder)
    os.mkdir(enrichr_plots_folder)

    for i in range(len(enrs)):
        try:
            gseapy.barplot(enrs[i].results, group="Gene_set", color=['darkred', 'darkblue'])
            ax = plt.gca()
            ax.set_facecolor("white")
            plt.savefig(os.path.join(enrichr_plots_folder, "enrichr_topic_{0}.png".format(i)), transparent=False, bbox_inches="tight")
        except:
            pass
        
plot_enrichr(enrs_0, 5)
plot_enrichr(enrs_1, 10)
plot_enrichr(enrs_2, 25)

## Predictive models

In [13]:
fht = pd.read_csv(os.path.join(RESULTS_DIR, output_folder, "FragmentHasTopic.csv"))
smiles_list = fht["smiles"].tolist()
W = np.array(fht[list(fht.columns)[5:]])

In [14]:
def binarize_by_cutoff(W, cutoff=0.33, max_pos=0.1):
    O = np.zeros(W.shape, dtype=int)
    O[W > cutoff] = 1
    max_pos = int(W.shape[0]*max_pos)
    for i in range(O.shape[1]):
        n = np.sum(O[:,i])
        if n > max_pos:
            idxs = np.argsort(W[:,i])[::-1][:max_pos]
            O[:,i] = 0
            O[idxs,i] = 1
    return O

In [15]:
from fragmentembedding import FragmentEmbedder
from miniautoml.binary_classifier import train_binary_classifier

Y = binarize_by_cutoff(W)
R = []
for i in range(len(fids)):
    r = [fids[i], fid2smi[fids[i]]] + [y_ for y_ in Y[i,:]]
    R += [r]
ds = pd.DataFrame(R, columns=["fid", "smiles"]+["signature_{0}".format(i) for i in range(NUM_TOPICS)])
ds.to_csv(os.path.join(RESULTS_DIR, output_folder, "fragment_signatures.csv"), index=False)

smiles_list = ds["smiles"].tolist()

X = FragmentEmbedder().transform(smiles_list)

mdl_folder = os.path.join(RESULTS_DIR, output_folder, "modeling_validations")
if os.path.exists(mdl_folder):
    shutil.rmtree(mdl_folder)
os.mkdir(mdl_folder)

validation_metrics_all_topics = []
y_columns = list(ds.columns[2:])
for yc in y_columns:
    y = ds[yc].tolist()
    mdl = train_binary_classifier(X, y, n_splits=10)
    print(yc, mdl.validation_metrics["aucs"])
    joblib.dump(mdl, os.path.join(mdl_folder, yc+".joblib"))
    validation_metrics_all_topics += [mdl.validation_metrics]

joblib.dump(validation_metrics_all_topics, os.path.join(mdl_folder, "validation_metrics.joblib"))

np.save(os.path.join(RESULTS_DIR, output_folder, "modeling_validations", "X.npy"), X)
np.save(os.path.join(RESULTS_DIR, output_folder, "modeling_validations", "Y.npy"), Y)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 412/412 [00:05<00:00, 78.91it/s]


Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
signature_0 [0.7461538461538462, 0.7692307692307693, 0.7461538461538462, 0.8974358974358975, 0.8410256410256409, 0.8871794871794872, 0.5871794871794872, 0.7717948717948718, 0.823076923076923, 0.4179487179487179]
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
signature_1 [0.8433333333333332, 0.7683333333333333, 0.8016666666666666, 0.7849999999999999, 0.75, 0.6933333333333334, 0.8283333333333333, 0.695, 0.71, 0.705]
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
signature_2 [0.9333333333333333, 0.9966666666666667, 0.9266666666666666, 0.9616666666666667, 0.9083333333333334, 0.9933333333333334, 0.9483333333333333, 0.9650000000000001, 0.99, 0.9583333333333334]
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
signature_3 [0.6699999999999999, 0.6016666666666667, 0

In [16]:
df = pd.read_csv("../data/enamine_stock.csv")
ds = pd.read_csv(os.path.join(RESULTS_DIR, output_folder, "fragment_signatures.csv"))
headers = list(ds.columns)[2:]

smiles_list = df["smiles"].tolist()
identifiers = df["catalog_id"].tolist()
X = FragmentEmbedder().transform(smiles_list)

R = []
for h in headers:
    print(h)
    mdl = joblib.load(os.path.join(mdl_folder, h+".joblib"))
    y_hat = list(mdl.predict(X))
    R += [y_hat]
Y_hat = np.array(R).T

R = []
for i in range(Y_hat.shape[0]):
    r = [identifiers[i], smiles_list[i]] + [y_ for y_ in Y_hat[i,:]]
    R += [r]

df = pd.DataFrame(R, columns = ["catalog_id", "smiles"] + headers)
df.to_csv(os.path.join(mdl_folder, "enamine_predictions.csv"), index=False)

/Users/mduranfrigola/miniconda3/envs/topicmodel/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning:

Trying to unpickle estimator VarianceThreshold from version 0.23.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

/Users/mduranfrigola/miniconda3/envs/topicmodel/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning:

Trying to unpickle estimator VarianceThreshold from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

/Users/mduranfrigola/miniconda3/envs/topicmodel/lib/python3.10/site-packages/sklearn/base.py:299: UserWarning:

Trying to unpickle estimator KBinsDiscretizer from version 1.2.2 when using versi

signature_0
signature_1
signature_2
signature_3
signature_4
signature_5
signature_6
signature_7
signature_8
signature_9


In [17]:
%%capture

import stylia
fig, axs = stylia.create_figure(1,1)
ax = axs.next()
for i, vd in enumerate(validation_metrics_all_topics):
    vals = np.array(vd["aucs"])
    noise = np.random.normal(0, 0.1, size=len(vals))
    ax.scatter(noise+i, vals)
ax.set_xticks([i for i in range(NUM_TOPICS)])
stylia.label(ax, title="ROC validation", xlabel="Topics", ylabel="AUROC")
    
stylia.save_figure(os.path.join(RESULTS_DIR, output_folder, "modeling_validations", "aucs.png"))

In [18]:
R = []
for i, vd in enumerate(validation_metrics_all_topics):
    r = ["Topic {0}".format(i), vd["positives"], np.mean(vd["aucs"]), np.std(vd["aucs"]), np.min(vd["aucs"]), np.max(vd["aucs"])]
    R += [r]
pd.DataFrame(R, columns=["topic", "positives", "auroc_mean", "auroc_std", "auroc_min", "auroc_max"]).to_csv(os.path.join(mdl_folder, "validation_summary.csv"), index=False)

## Enrichment figure

In [19]:
import os
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
import collections
import matplotlib.pyplot as plt
import stylia
from stylia.colors import ContinuousColorMap
import joblib
import seaborn as sns

data_dir = os.path.join(RESULTS_DIR, output_folder)

top_proteins = [5, 10, 25]

gs_inds = {
    "gs_ind_0": "CC",
    "gs_ind_1": "MF"
}

R = []
for t in top_proteins:
    for i in range(NUM_TOPICS):
        s = "topic_{0}.csv".format(i)
        file_name = os.path.join(data_dir, "enrichr_results_{0}".format(t), "topic_{0}.csv".format(i))
        df = pd.read_csv(file_name)
        df = df[df["Adjusted P-value"] < 0.01]
        for v in df.values:
            gs = gs_inds[v[0]]
            term = v[1]
            over = int(v[2].split("/")[0])
            total = int(v[2].split("/")[1])
            pval = -np.log10(v[3])
            adj_pval = -np.log10(v[4])
            odds = v[5]
            genes = v[6].split(";")
            topic = "Topic {0}".format(i)
            R += [[t, gs, term, topic, over, total, pval, adj_pval, odds, genes]]
            
unique_terms = sorted(set([(r[1], r[2]) for r in R]))
ut_idxs = dict((k,i) for i,k in enumerate(unique_terms))
topic_names = ["Topic {0}".format(i) for i in range(NUM_TOPICS)]
topic_tops = [(tn, t) for tn in topic_names for t in top_proteins]
tt_idxs = dict((k,i) for i,k in enumerate(topic_tops))

M = np.zeros((len(unique_terms), len(topic_tops)))
for r in R:
    i = ut_idxs[(r[1], r[2])]
    j = tt_idxs[(r[3], r[0])]
    v = r[7]
    M[i,j] = v
    
M[M > 10] = 10
M[M < 2] = 0

Z = hierarchy.ward(M)
hierarchy.leaves_list(Z)
clustered_idxs = hierarchy.leaves_list(hierarchy.optimal_leaf_ordering(Z, M))

unique_terms = [unique_terms[idx] for idx in clustered_idxs]
M = M[clustered_idxs]

term_genes = {}
for ut in unique_terms:
    d = collections.defaultdict(int)
    for r in R:
        k = (r[1], r[2])
        if k == ut:
            for g in r[-1]:
                d[g] += 1
    genes = [x[0] for x in sorted(d.items(), key=lambda x: -x[1])]
    term_genes[ut] = genes

In [20]:
%%capture

cmap = ContinuousColorMap()
cmap.fit([i for i in range(len(topic_names))])
topic_colors = cmap.transform([i for i in range(len(topic_names))])
topic_color_dict = dict((topic_names[i], c) for i,c in enumerate(topic_colors))

fig, axs = stylia.create_figure(1,1, width=15, height=20)
ax = axs.next()

xs = []
ys = []
ss = []
cs = []

for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        v = M[i,j]
        xs += [j]
        ys += [i]
        ss += [v*10]
        cs += [topic_color_dict[topic_tops[j][0]]]
ax.scatter(xs, ys, s=ss, color=cs)
xticks = []
xticklabels = []
for i, t in enumerate(topic_tops):
    xticks += [i]
    if t[1] == top_proteins[1]:
        xticklabels += [t[0].split(" ")[1]]
    else:
        xticklabels += [""]
yticks = [i for i in range(M.shape[0])]
yticklabels = ["{0} [{1}]".format(t[1], t[0]) for t in unique_terms]
yticklabels_right = [" ".join(term_genes[t][:10]) for t in unique_terms]
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_ylim(-1, M.shape[0])
ax.set_xlim(-1, M.shape[1])
ax.set_xticklabels(xticklabels)
ax.set_yticklabels(yticklabels)
ax_right = ax.twinx()
ax_right.set_yticks(yticks)
ax_right.set_yticklabels(yticklabels_right)
ax_right.set_ylim(-1, M.shape[0])
ax_right.grid(visible=False)

stylia.label(ax, title="Enrichment analysis of topics", xlabel = "Topics", ylabel = "")
plt.tight_layout()

stylia.save_figure(os.path.join(data_dir, "enrichr_matrix.png"))

## Suggest topic names

In [21]:
enr_scores = collections.defaultdict(list)

for i in range(NUM_TOPICS):
    for n in [5, 10, 25]:
        enr = pd.read_csv(os.path.join(RESULTS_DIR, output_folder, "enrichr_results_{0}".format(n), "topic_{0}.csv".format(i)))
        for r in enr.values:
            k = (i, r[0], r[1])
            pval = r[3]
            if pval > 0.05:
                continue
            v = -np.log10(pval)
            #v = r[5]
            enr_scores[k] += [v]

agg_enr_scores = collections.defaultdict(list)

for k,v in enr_scores.items():
    agg_enr_scores[k[0]] += [(k[1], k[2], np.max(v))]

topic_annotations = {}
for k,v in agg_enr_scores.items():
    v = sorted(v, key=lambda x: -x[2])
    topic_annotations[k] = v[:5]
    
R = []
for k,v in topic_annotations.items():
    for x in v:
        R += [[k, x[0], x[1], x[2]]]
        
enr = pd.DataFrame(R, columns=["topic", "go_cat", "term", "logpval_sum"])

enr.to_csv(os.path.join(RESULTS_DIR, output_folder, "enrichr_summary_results.csv"), index=False)